In [1]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]


In [2]:
editDistance(x="GCGTATGC", y="TATTGGCTATACGGTT")

11

Adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [3]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

Q1. What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1?

Q2. What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1?

In [4]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [12]:
genome_filename = 'chr1.GRCh38.excerpt.fasta'

t = readGenome(genome_filename)

# Question1
p = 'GCTGATCGATCGTACG'
print(editDistance(t, p))

799984


In [13]:
# Question2
p = 'GATTTACCAGATTGAG'
print(editDistance(t, p))

799984


Find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. Picture the overlap graph corresponding to the overlaps just calculated.

Q3. How many edges are in the graph? In other words, how many distinct pairs of reads overlap?

In [7]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match


def smart_overlap_map(reads, k):
    olaps = {}
    result = {}
    for read in reads:
        for i in range(len(read)-k+1):
            if read[i:i+k] not in olaps:
                olaps[read[i:i+k]] = [read]
            else:
                olaps[read[i:i+k]].append(read)

    count = 0
    for read in reads:
        read_suffix = read[-k:]
        for possible_read in olaps[read_suffix]:
            if possible_read != read:
                olen = overlap(read, possible_read, k)
                if olen > 0:
                    count += 1
                    result[(read, possible_read)] = olen

    return result, count

In [10]:
reads_filename = 'ERR266411_1.for_asm.fastq'
reads, _ = readFastq(reads_filename)
pairs_olen, pairs_count = smart_overlap_map(reads, 30)

# Question 3
print("Number of pairs of reads overlap:", pairs_count)

Number of pairs of reads overlap: 904746


Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least 
k
k bases long. To make an overlap graph, we could call 
overlap(a, b, min_length=k)
overlap(a, b, min_length=k) on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read 
a
a whose length-6 suffix is 
GTCCTA
GTCCTA.  Say 
GTCCTA
GTCCTA does not occur in any other read in the dataset.  In other words, the 6-mer 
GTCCTA
GTCCTA occurs at the end of read 
a
a and nowhere else.  It follows that 
a
a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read 
a
a and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of 
a
a.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea.  You don't have to do it this way, but this might help you.  Let every k-mer in the dataset have an associated 
Python 
set
set
 object
, which starts out empty.  We use a 
Python dictionary
 to associate each k-mer with its corresponding 
set
set. (1) For every k-mer in a read, we add the read to the 
set
set object corresponding to that k-mer.  If our read is 
GATTA
GATTA and k=3, we would add 
GATTA
GATTA to the 
set
set objects for 
GAT
GAT, 
ATT
ATT and 
TTA
TTA.  We do this for every read so that, at the end, each 
set
set contains all reads containing the corresponding k-mer.  (2) Now, for each read 
a
a, we find all overlaps involving a suffix of 
a
a.  To do this, we take 
a
a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding 
set
set) and call 
overlap(a, b, min_length=k)
overlap(a, b, min_length=k) for each.

The most important point is that we do not call 
overlap(a, b, min_length=k)
overlap(a, b, min_length=k) if 
b
b does not contain the length-k suffix of 
a
a.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.

Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.

Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called 
overlap(a, b, min_length=k)
overlap(a, b, min_length=k) on every pair of reads.  We also have provided 
a couple examples you can check against
.

Picture the overlap graph corresponding to the overlaps just calculated.  How many edges are in the graph?  In other words, how many distinct pairs of reads overlap?

In [11]:
# Question 4
reads_involved = []
for key, value in pairs_olen:
    reads_involved.append(key)
print("Number of reads have a suffix involved in an overlap:", len(set(reads_involved)))

Number of reads have a suffix involved in an overlap: 7161
